<a href="https://colab.research.google.com/github/anamanuellar/iagents-jur/blob/main/Desafio_4_VR_Agent_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [462]:
import pandas as pd
import openpyxl
import xlsxwriter
import holidays
import numpy as np

In [463]:
from datetime import datetime

In [557]:
# Carregar todas as bases
ativos = pd.read_excel("/content/automacao_vr/ATIVOS.xlsx")
ferias = pd.read_excel("/content/automacao_vr/FÉRIAS.xlsx")
afastamentos = pd.read_excel("/content/automacao_vr/AFASTAMENTOS.xlsx")
estagiarios = pd.read_excel("/content/automacao_vr/ESTÁGIO.xlsx")
aprendizes = pd.read_excel("/content/automacao_vr/APRENDIZ.xlsx")
exterior = pd.read_excel("/content/automacao_vr/EXTERIOR.xlsx")
desligados = pd.read_excel("/content/automacao_vr/DESLIGADOS.xlsx")
admissoes = pd.read_excel("/content/automacao_vr/ADMISSÃO ABRIL.xlsx")
dias_uteis = pd.read_excel("/content/automacao_vr/Base dias uteis.xlsx")
valor_vr = pd.read_excel("/content/automacao_vr/Base sindicato x valor.xlsx")

In [558]:
# Normaliza nomes de colunas (minúsculas, _)
def norm_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        pd.Index(df.columns)
        .map(str)
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.lower()
    )
    return df

# Converte colunas para date (ou NaT)
def to_date(df: pd.DataFrame, cols) -> pd.DataFrame:
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce").dt.date
    return df

# Trim + upper para campos textuais críticos
def trim_upper(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.replace(r"[\u200B-\u200D\uFEFF]", "", regex=True) # remove invisíveis
         .str.strip()
         .str.upper()
         .replace({"NAN": np.nan})
    )

In [559]:
# Férias

def preprocess_ferias(df: pd.DataFrame) -> pd.DataFrame:
    df = norm_cols(df)
    # Aceita nomes diferentes e renomeia
    rename_map = {
        "inicio": "inicio_ferias",
        "início": "inicio_ferias",
        "fim": "fim_ferias",
    }
    df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
    # Datas
    df = to_date(df, ["inicio_ferias","fim_ferias"])
    # Mantém apenas colunas necessárias
    keep = ["matricula","inicio_ferias","fim_ferias"]
    return df[[c for c in keep if c in df.columns]].dropna(subset=["matricula"])


In [560]:
# Desligados

def preprocess_desligados(df: pd.DataFrame) -> pd.DataFrame:
    df = norm_cols(df)
    # Ajusta nomes comuns
    df = df.rename(columns={
        "deslig_ok":"deslig_ok",
        "ok":"deslig_ok",
        "status":"deslig_ok",
        "data_informacao":"data_informada"
    })
    df["deslig_ok"] = trim_upper(df.get("deslig_ok", pd.Series(index=df.index)))
    df = to_date(df, ["data_desligamento","data_informada"])
    keep = ["matricula","data_desligamento","deslig_ok","data_informada"]
    return df[[c for c in keep if c in df.columns]].dropna(subset=["matricula"])


In [561]:
# Admissões

def preprocess_admissoes(df: pd.DataFrame) -> pd.DataFrame:
    df = norm_cols(df)
    df = df.rename(columns={"admissao":"data_admissao"})
    df = to_date(df, ["data_admissao"])
    keep = ["matricula","data_admissao"]
    return df[[c for c in keep if c in df.columns]].dropna(subset=["matricula"])


In [562]:
# Sindicato x Valor

def preprocess_sindicato_valor(df: pd.DataFrame) -> pd.DataFrame:
    df = norm_cols(df)

    # Renomeia colunas para padrão
    if "sindicado" in df.columns:
        df = df.rename(columns={"sindicado": "sindicato"})
    if "estado" in df.columns:
        df = df.rename(columns={"estado": "estado"})  # já padroniza nome

    # Normaliza texto
    if "sindicato" in df.columns:
        df["sindicato"] = trim_upper(df["sindicato"])
    if "estado" in df.columns:
        df["estado"] = df["estado"].str.strip()

    # Trata valor_vr_dia
    if "valor_vr_dia" in df.columns:
        df["valor_vr_dia"] = (
            df["valor_vr_dia"]
              .astype(str).str.replace(",", ".", regex=False)
              .astype(float)
        )

    # Mantém só as colunas relevantes
    if "estado" in df.columns:
        keep = ["estado", "valor_vr_dia"]
    else:
        keep = ["matricula", "sindicato", "valor_vr_dia"]

    cols = [c for c in keep if c in df.columns]
    return df[cols].dropna(subset=[cols[0]], how="any")



In [563]:
dias_uteis = pd.read_excel("/content/automacao_vr/Base dias uteis.xlsx", header=1)
dias_uteis.columns = dias_uteis.columns.str.strip().str.lower().str.replace(" ", "_")

In [564]:
# Dias Úteis

def preprocess_dias_uteis_sindicato(df: pd.DataFrame) -> pd.DataFrame:
    # Padroniza colunas
    df = norm_cols(df)

    # Garante que o nome da coluna sindicato esteja correto
    if "sindicado" in df.columns:
        df = df.rename(columns={"sindicado": "sindicato"})

    # Remove espaços invisíveis e coloca texto em maiúsculas
    df["sindicato"] = trim_upper(df["sindicato"])

    # Garante que dias_uteis seja numérico
    df["dias_uteis"] = pd.to_numeric(df["dias_uteis"], errors="coerce")

    return df[["sindicato", "dias_uteis"]].dropna()


In [565]:
# Exterior

def preprocess_exterior(df: pd.DataFrame) -> pd.DataFrame:
    # Padroniza nomes das colunas
    df.columns = df.columns.str.strip().str.lower()

    # Garante que existe a coluna 'cadastro'
    if "cadastro" not in df.columns:
        raise KeyError(f"Coluna 'cadastro' não encontrada. Colunas disponíveis: {df.columns}")

    # Cria DataFrame apenas com matrícula (cadastro) e flag
    df_out = df[["cadastro"]].dropna().drop_duplicates()
    df_out = df_out.rename(columns={"cadastro": "matricula"})
    df_out["flag_exterior"] = True

    return df_out


In [566]:
estagiarios.columns

Index(['MATRICULA', 'TITULO DO CARGO', 'na compra?'], dtype='object')

In [567]:
# Estágio / Aprendiz

def preprocess_listas_excluir(df: pd.DataFrame, flag_col: str) -> pd.DataFrame:
    # Converte os nomes de colunas para minúsculo e sem espaços extras
    df.columns = df.columns.str.strip().str.lower()
    # Agora seleciona a coluna 'matricula'
    return df[["matricula"]].assign(**{flag_col: True}).dropna(subset=["matricula"])

# Uso:
estagios_pp   = preprocess_listas_excluir(estagiarios, "flag_estagio")
aprendizes_pp = preprocess_listas_excluir(aprendizes,  "flag_aprendiz")




In [568]:
# Ativos

def preprocess_ativos(df: pd.DataFrame) -> pd.DataFrame:
    df = norm_cols(df)
    # renomeia campos comuns
    df = df.rename(columns={
        "titulo_do_cargo":"cargo",
        "uf":"local_trabalho_uf",
        "estado":"local_trabalho_uf",
        "municipio":"local_trabalho_municipio",
        "cidade":"local_trabalho_municipio",
        "admissao":"data_admissao",
        "desligamento":"data_desligamento"
    })
    # datas
    df = to_date(df, ["data_admissao","data_desligamento","afastamento_inicio","afastamento_fim"])
    # texto
    for c in ["cargo","sindicato","local_trabalho_uf","local_trabalho_municipio","pais"]:
        if c in df.columns:
            df[c] = trim_upper(df[c])
    # flags de exclusão por cargo (diretor, estagiário, aprendiz)
    if "cargo" in df.columns:
        cargo = df["cargo"].fillna("")
        df["flag_excl_cargo"] = (
            cargo.str.contains("DIRETOR", na=False)
            | cargo.str.contains("ESTAG",  na=False)  # ESTAGIÁRIO
            | cargo.str.contains("APREND", na=False)
        )
    else:
        df["flag_excl_cargo"] = False

    # país/UF exterior (se houver)
    if "pais" in df.columns:
        df["flag_exterior"] = df["pais"].eq("EX") | df["pais"].ne("BR")
    elif "local_trabalho_uf" in df.columns:
        df["flag_exterior"] = df["local_trabalho_uf"].eq("EX")
    else:
        df["flag_exterior"] = False

    # mantém campos chave para os próximos passos
    keep = [
        "matricula","data_admissao","nome","cargo","sindicato",
        "local_trabalho_uf","local_trabalho_municipio","pais",
        "data_admissao","data_desligamento",
        "afastamento_inicio","afastamento_fim",
        "flag_excl_cargo","flag_exterior"
    ]
    keep = [c for c in keep if c in df.columns]
    return df[keep].dropna(subset=["matricula"])


In [569]:
# Pré processamento

ferias_pp        = preprocess_ferias(ferias)
desligados_pp    = preprocess_desligados(desligados)
admissoes_pp     = preprocess_admissoes(admissoes)
valor_vr_pp = preprocess_sindicato_valor(valor_vr)
dias_uteis_pp = preprocess_dias_uteis_sindicato(dias_uteis)
ativos_pp        = preprocess_ativos(ativos)
exterior_pp      = preprocess_exterior(exterior)



# Mescla flags de exclusão por matrícula
from functools import reduce
flags_listas = [estagios_pp, aprendizes_pp, exterior_pp]
flags = reduce(lambda L,R: pd.merge(L, R, on="matricula", how="outer"), flags_listas)
for c in ["flag_estagio","flag_aprendiz","flag_exterior"]:
    if c not in flags.columns: flags[c] = False

# Leva flags para ATIVOS
ativos_pp = ativos_pp.merge(flags, on="matricula", how="left")
for c in ["flag_estagio","flag_aprendiz","flag_exterior"]:
    if c in ativos_pp.columns:
        ativos_pp[c] = ativos_pp[c].fillna(False)

print("Pré-processamento concluído:")
print("ativos_pp:", ativos_pp.shape, "| ferias_pp:", ferias_pp.shape,
      "| desligados_pp:", desligados_pp.shape, "| admissoes_pp:", admissoes_pp.shape,
      "| valor_vr_pp:", valor_vr_pp.shape)


Pré-processamento concluído:
ativos_pp: (1815, 8) | ferias_pp: (80, 1) | desligados_pp: (51, 2) | admissoes_pp: (83, 1) | valor_vr_pp: (5, 1)


/tmp/ipython-input-2939865554.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"NAN": np.nan})
/tmp/ipython-input-3733892367.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ativos_pp[c] = ativos_pp[c].fillna(False)


In [570]:
# Afastamentos

def preprocess_afastamentos(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.strip().str.lower()
    if "matricula" not in df.columns:
        raise KeyError(f"Coluna 'matricula' não encontrada. Colunas disponíveis: {df.columns}")
    return df[["matricula"]].assign(flag_afastado=True).dropna(subset=["matricula"])

afastamentos_pp = preprocess_afastamentos(afastamentos)


In [571]:
from functools import reduce

# Lista de DataFrames de exclusão
flags_listas = [estagios_pp, aprendizes_pp, exterior_pp]

# Junta todas as flags pelo campo 'matricula'
flags = reduce(lambda left, right: pd.merge(left, right, on="matricula", how="outer"), flags_listas)

# Preenche valores NaN com False
flags = flags.fillna(False)

# Merge das flags com a base de Ativos
ativos_filtrado = ativos_pp.merge(flags, on="matricula", how="left")

# Marca valores NaN como False
for col in ["flag_estagio", "flag_aprendiz", "flag_exterior"]:
    if col in ativos_filtrado.columns:
        ativos_filtrado[col] = ativos_filtrado[col].fillna(False)

# Unificar após o merge
for col in ["flag_estagio", "flag_aprendiz", "flag_exterior"]:
    col_x = f"{col}_x"
    col_y = f"{col}_y"
    if col_x in ativos_filtrado.columns and col_y in ativos_filtrado.columns:
        ativos_filtrado[col] = ativos_filtrado[col_x].fillna(ativos_filtrado[col_y])
    elif col_x in ativos_filtrado.columns:
        ativos_filtrado[col] = ativos_filtrado[col_x]
    elif col_y in ativos_filtrado.columns:
        ativos_filtrado[col] = ativos_filtrado[col_y]
    else:
        ativos_filtrado[col] = False

# Exclui cargos específicos
excluir_cargos = ["DIRETOR", "ESTAGIÁRIO", "APRENDIZ"]
ativos_filtrado = ativos_filtrado[~ativos_filtrado["cargo"].str.upper().isin(excluir_cargos)]

# Exclui afastados (caso tenha base de afastamentos)
if 'afastamentos_pp' in globals():
    ativos_filtrado = ativos_filtrado.merge(afastamentos_pp, on="matricula", how="left")
    if "flag_afastado" in ativos_filtrado.columns:
        ativos_filtrado = ativos_filtrado[ativos_filtrado["flag_afastado"] != True]

# Exclui quem está em qualquer uma das flags
ativos_filtrado = ativos_filtrado[
    ~(ativos_filtrado["flag_estagio"] | ativos_filtrado["flag_aprendiz"] | ativos_filtrado["flag_exterior"])
]

print(f"Base final após exclusões: {len(ativos_filtrado)} colaboradores")



Base final após exclusões: 1795 colaboradores


/tmp/ipython-input-3639805988.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  flags = flags.fillna(False)
/tmp/ipython-input-3639805988.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ativos_filtrado[col] = ativos_filtrado[col].fillna(False)


In [572]:
# Leitura já renomeando
valor_vr_pp = pd.read_excel(
    "/content/automacao_vr/Base sindicato x valor.xlsx",
    header=0
)

# Renomeia as colunas manualmente
valor_vr_pp.columns = ["estado", "valor_vr_dia"]

# Remove linhas em branco
valor_vr_pp = valor_vr_pp.dropna(subset=["estado", "valor_vr_dia"])

valor_vr_pp



,estado,valor_vr_dia
0,Paraná,35.0
1,Rio de Janeiro,35.0
2,Rio Grande do Sul,35.0
3,São Paulo,37.5


In [573]:
dias_uteis_pp

,sindicato,dias_uteis
0,SITEPD PR - SIND DOS TRAB EM EMPR PRIVADAS DE ...,22
1,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,21
2,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,22
3,SINDPD RJ - SINDICATO PROFISSIONAIS DE PROC DA...,21


In [574]:
# ============================
# ETAPA FINAL - CÁLCULO VR COM PADRONIZAÇÃO
# ============================


import unicodedata

# --- 1️⃣ Função para padronizar estados ---
def normalizar_estado(s):
    if pd.isna(s):
        return s
    s = str(s).strip()
    s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('utf-8')
    return s.title()  # Ex.: "parana" ou "Paraná" -> "Parana"

In [575]:
# --- 2️⃣ Padronização de colunas ---
def padronizar_cols(df):
    return df.rename(columns=lambda x: str(x).strip())

ativos_filtrado = padronizar_cols(ativos_filtrado)
valor_vr_pp     = padronizar_cols(valor_vr_pp)
dias_uteis_pp   = padronizar_cols(dias_uteis_pp)


In [576]:
# --- 3️⃣ Garantir nomes corretos ---
# Dias úteis
col_dias = [c for c in dias_uteis_pp.columns if "dias" in c.lower() and "uteis" in c.lower()]
if col_dias:
    dias_uteis_pp.rename(columns={col_dias[0]: "dias_uteis_mes"}, inplace=True)
else:
    raise KeyError(f"Coluna de dias úteis não encontrada: {dias_uteis_pp.columns}")

# Valor VR
col_valor = [c for c in valor_vr_pp.columns if "valor" in c.lower()]
if col_valor:
    valor_vr_pp.rename(columns={col_valor[0]: "valor_vr_dia"}, inplace=True)
else:
    raise KeyError(f"Coluna de valor não encontrada: {valor_vr_pp.columns}")

In [577]:
# --- 4️⃣ Merge com dias úteis (por sindicato) ---
base_calc = ativos_filtrado.merge(dias_uteis_pp, on="sindicato", how="left")

In [578]:
# --- 5️⃣ Criar coluna 'estado' com mapeamento sindicato -> estado ---
mapa_sindicato_estado = {
    "SINDPD RJ - SINDICATO PROFISSIONAIS DE PROC DADOS DO RIO DE JANEIRO": "Rio de Janeiro",
    "SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE DADOS RIO GRANDE DO SUL": "Rio Grande do Sul",
    "SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMPRESAS PROC DADOS ESTADO DE SP.": "São Paulo",
    "SITEPD PR - SIND DOS TRAB EM EMPR PRIVADAS DE PROC DE DADOS DE CURITIBA E REGIAO METROPOLITANA": "Paraná"
}
base_calc["estado"] = base_calc["sindicato"].replace(mapa_sindicato_estado)


In [579]:
# --- 6️⃣ Padroniza estados nas duas tabelas ---
valor_vr_pp["estado"] = valor_vr_pp["estado"].map(normalizar_estado)
base_calc["estado"]   = base_calc["estado"].map(normalizar_estado)

In [580]:
# --- 7️⃣ Merge com valor VR (por estado) ---
base_calc = base_calc.merge(valor_vr_pp, on="estado", how="left")

In [581]:
# --- 8️⃣ Regra de dias proporcionais ---
def calcular_dias_proporcionais(row):
    if pd.isnull(row.get("data_desligamento")):
        return row["dias_uteis_mes"]
    dia_desligamento = row["data_desligamento"].day
    if dia_desligamento <= 15:
        return 0
    else:
        return min(dia_desligamento, row["dias_uteis_mes"])

base_calc["dias_vr"] = base_calc.apply(calcular_dias_proporcionais, axis=1)

In [582]:
# --- 9️⃣ Cálculo do VR ---
base_calc["valor_total_vr"] = base_calc["dias_vr"] * base_calc["valor_vr_dia"]
base_calc["empresa_paga"] = base_calc["valor_total_vr"] * 0.8
base_calc["desconto_colaborador"] = base_calc["valor_total_vr"] * 0.2

In [585]:
if "data_admissao" not in base_calc.columns:
    base_calc["data_admissao"] = ""

In [589]:
# Criar coluna competência fixa
base_calc["competencia"] = "05/2025"

In [591]:
# Criar coluna competência fixa
base_calc["Observação"] = ""

In [592]:
base_calc

,matricula,cargo,sindicato,flag_excl_cargo,flag_exterior_x,flag_estagio_x,flag_aprendiz_x,flag_exterior_y,flag_estagio_y,flag_aprendiz_y,...,dias_uteis_mes,estado,valor_vr_dia,dias_vr,valor_total_vr,empresa_paga,desconto_colaborador,data_admissao,competencia,Observação
0,34941,TECH RECRUITER II,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,False,False,False,False,NaN,NaN,NaN,...,22,Sao Paulo,37.5,22,825.0,660.0,165.0,,05/2025,
1,24401,COORDENADOR ADMINISTRATIVO,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,
2,32104,ANALISTA CONTABIL-FISCAL II,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,
3,35254,ANALISTA CONTABIL-FISCAL II,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,
4,31731,ANALISTA CONTABIL-FISCAL III,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,32050,ANALISTA DE SISTEMAS E NEGOCIOS II,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,
1791,32549,ANALISTA DE SISTEMAS E NEGOCIOS II,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,
1792,31372,ANALISTA DE SISTEMAS E NEGOCIOS III,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,
1793,31573,ANALISTA DE SISTEMAS E NEGOCIOS III,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,False,False,False,False,NaN,NaN,NaN,...,21,Rio Grande Do Sul,35.0,21,735.0,588.0,147.0,,05/2025,


In [595]:
# --- 🔟 Resultado final ---
resultado_vr = base_calc[
    ["matricula",             # Matrícula
     "data_admissao",         # Admissão
     "sindicato",             # Sindicato do Colaborador
     "competencia",           # Competência
     "dias_vr",               # Dias
     "valor_vr_dia",          # VALOR DIÁRIO VR
     "valor_total_vr",        # TOTAL
     "empresa_paga",          # Custo empresa
     "desconto_colaborador",   # Desconto profissional
     "Observação"             # Observação
    ]
].sort_values(by="matricula")

print("Prévia do resultado final:")
resultado_vr.head()

Prévia do resultado final:


,matricula,data_admissao,sindicato,competencia,dias_vr,valor_vr_dia,valor_total_vr,empresa_paga,desconto_colaborador,Observação
410,20792,,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,05/2025,21,35.0,735.0,588.0,147.0,
1190,21827,,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,05/2025,22,37.5,825.0,660.0,165.0,
32,23499,,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,05/2025,21,35.0,735.0,588.0,147.0,
1579,23836,,SINDPD RJ - SINDICATO PROFISSIONAIS DE PROC DA...,05/2025,21,35.0,735.0,588.0,147.0,
7,24122,,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,05/2025,21,35.0,735.0,588.0,147.0,


In [596]:
# --- 📤 Exportar resultado final ---
caminho_arquivo = "/content/resultado_vr5.xlsx"  # ajuste o caminho se precisar

resultado_vr.to_excel(caminho_arquivo, index=False)

print(f"Arquivo exportado com sucesso para: {caminho_arquivo}")

Arquivo exportado com sucesso para: /content/resultado_vr5.xlsx
